# Action Recognition @ UCF101  
**Due date: 11:59 pm on Nov. 19, 2019 (Tuesday)**

## Description
---
In this homework, you will be doing action recognition using Recurrent Neural Network (RNN), (Long-Short Term Memory) LSTM in particular. You will be given a dataset called UCF101, which consists of 101 different actions/classes and for each action, there will be 145 samples. We tagged each sample into either training or testing. Each sample is supposed to be a short video, but we sampled 25 frames from each videos to reduce the amount of data. Consequently, a training sample is an image tuple that forms a 3D volume with one dimension encoding *temporal correlation* between frames and a label indicating what action it is.

To tackle this problem, we aim to build a neural network that can not only capture spatial information of each frame but also temporal information between frames. Fortunately, you don't have to do this on your own. RNN — a type of neural network designed to deal with time-series data — is right here for you to use. In particular, you will be using LSTM for this task.

Instead of training an end-to-end neural network from scratch whose computation is prohibitively expensive, we divide this into two steps: feature extraction and modelling. Below are the things you need to implement for this homework:
- **{35 pts} Feature extraction**. Use any of the [pre-trained models](https://pytorch.org/docs/stable/torchvision/models.html) to extract features from each frame. Specifically, we recommend not to use the activations of the last layer as the features tend to be task specific towards the end of the network. 
    **hints**: 
    - A good starting point would be to use a pre-trained VGG16 network, we suggest first fully connected layer `torchvision.models.vgg16` (4096 dim) as features of each video frame. This will result into a 4096x25 matrix for each video. 
    - Normalize your images using `torchvision.transforms` 
    ```
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    prep = transforms.Compose([ transforms.ToTensor(), normalize ])
    prep(img)
    The mean and std. mentioned above is specific to Imagenet data
    
    ```
    More details of image preprocessing in PyTorch can be found at http://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    
- **{35 pts} Modelling**. With the extracted features, build an LSTM network which takes a **dx25** sample as input (where **d** is the dimension of the extracted feature for each frame), and outputs the action label of that sample.
- **{20 pts} Evaluation**. After training your network, you need to evaluate your model with the testing data by computing the prediction accuracy **(5 points)**. The baseline test accuracy for this data is 75%, and **10 points** out of 20 is for achieving test accuracy greater than the baseline. Moreover, you need to compare **(5 points)** the result of your network with that of support vector machine (SVM) (stacking the **dx25** feature matrix to a long vector and train a SVM).
- **{10 pts} Report**. Details regarding the report can be found in the submission section below.

Notice that the size of the raw images is 256x340, whereas your pre-trained model might take **nxn** images as inputs. To solve this problem, instead of resizing the images which unfavorably changes the spatial ratio, we take a better solution: Cropping five **nxn** images, one at the image center and four at the corners and compute the **d**-dim features for each of them, and average these five **d**-dim feature to get a final feature representation for the raw image.
For example, VGG takes 224x224 images as inputs, so we take the five 224x224 croppings of the image, compute 4096-dim VGG features for each of them, and then take the mean of these five 4096-dim vectors to be the representation of the image.

In order to save you computational time, you need to do the classification task only for **the first 25** classes of the whole dataset. The same applies to those who have access to GPUs. **Bonus 10 points for running and reporting on the entire 101 classes.**


## Dataset
Download **dataset** at [UCF101](http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_images.tar)(Image data for each video) and the **annos folder** which has the video labels and the label to class name mapping is included in the assignment folder uploaded. 


UCF101 dataset contains 101 actions and 13,320 videos in total.  

+ `annos/actions.txt`  
  + lists all the actions (`ApplyEyeMakeup`, .., `YoYo`)   
  
+ `annots/videos_labels_subsets.txt`  
  + lists all the videos (`v_000001`, .., `v_013320`)  
  + labels (`1`, .., `101`)  
  + subsets (`1` for train, `2` for test)  

+ `images/`  
  + each folder represents a video
  + the video/folder name to class mapping can be found using `annots/videos_labels_subsets.txt`, for e.g. `v_000001` belongs to class 1 i.e. `ApplyEyeMakeup`
  + each video folder contains 25 frames  



## Some Tutorials
- Good materials for understanding RNN and LSTM
    - http://blog.echen.me
    - http://karpathy.github.io/2015/05/21/rnn-effectiveness/
    - http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Implementing RNN and LSTM with PyTorch
    - [LSTM with PyTorch](http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py)
    - [RNN with PyTorch](http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

In [4]:
# Mount your google drive where you've saved your assignment folder
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
# Set your working directory (in your google drive)
# Note that 'gdrive/My Drive/Y2019Fall/CSE-527-Intro-To-Computer-Vision/hw4' is just an example, 
#   change it to your specific homework directory.
cd '/content/gdrive/My Drive/Duppala_Sai_112684112_hw5'

/content/gdrive/My Drive/Duppala_Sai_112684112_hw5


In [0]:
# import packages here
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import random 
import time

from torchvision import datasets, models, transforms
# import os

# import cv2, subprocess
# import os, sys, collections, random, string
# # from scipy import ndimage
# from datetime import datetime
import torchvision.models as models
# from torchvision.transforms import ToTensor
# from PIL import Image
# import pickle
# from numpy import genfromtxt


import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader,sampler,Dataset
import torchvision.datasets as dset
import torchvision.transforms as T
import timeit
from PIL import Image
import os
import numpy as np
import scipy.io
import torchvision.models.inception as inception

from tqdm import tqdm
from torch.utils.data.sampler import SequentialSampler
from torch.utils.data.sampler import Sampler


# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [0]:
# !tar -xvf  '/content/drive/My Drive/Duppala_Sai_112684112_hw5/UCF101_images.tar' -C '/content/drive/My Drive/Duppala_Sai_112684112_hw5'

---
---
## **Problem 1.** Feature extraction

In [0]:
n_classes = 25
samples_per_class = 145

tot_folders = n_classes*samples_per_class


label_train = []
label_test = []

label_all = []

In [0]:
#get file names for training and testing
# bv
with open("./annos/videos_labels_subsets.txt", "r") as file:
    array = []
    for (i,line) in enumerate(file):
        if i < tot_folders:
            l = line[:-1].split('\t');
            label_all.append(l[1])


In [0]:
all_indices = []
all_labels = []
all_indicies_labels = []
for i,e in enumerate(label_all):
    if i % 15 == 0:
        for j in range(i*25,(i+1)*25):
            all_indices.append(j)
            all_indicies_labels.append(e)
        all_labels.append(e)


In [0]:
class ActionDataset(Dataset):
    
    def __init__(self,  root_dir,labels=[], transform=None):
        # root_dir (string): Directory with all the images.
        # labels (list): labels if images.
        # transform (callable, optional): Optional transform to be applied on a sample.

        self.root_dir = root_dir
        self.transform = transform
        # self.length=len(os.listdir(self.root_dir))
        #total number of folders - we dont need all for 25 classes
        self.length = tot_folders
        self.labels=labels

    def __len__(self):
        #3625*25 ?
        return self.length*25

    def __getitem__(self, idx):
        
        folder = int(idx/25+1)
        imidx = int(idx%25+1)


        folder = 'v_' + format(folder,'06d')
        imgname = 'i_' + format(imidx, '04d') + '.jpg'

        img_path = os.path.join(self.root_dir, folder,imgname)
        image = Image.open(img_path)

        if len(self.labels) !=0 :
            # Label = int(self.labels[int(idx/25)][0]) - 1
            Label = int(self.labels[int(idx/25)])

        if self.transform:
            image = self.transform(image)


        if len(self.labels)!=0:
            sample={'image':image,'img_path':img_path,'Label':Label}
        else:
            sample={'image':image,'img_path':img_path}
        return sample
    

In [0]:
#change this resize to proper resize
my_transform = transforms.Compose([transforms.Resize([224,224]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

image_dataset = ActionDataset(root_dir='./images/',labels=label_all,transform=my_transform)

image_dataset_sub15 = torch.utils.data.Subset(image_dataset, all_indices)

batch_size = 25
image_loader = torch.utils.data.DataLoader(
        image_dataset_sub15,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False,
    )

In [0]:
#Checking Images Manually
plt.imshow(image_dataset_sub15[6000]['image'].permute(1, 2, 0))
print('Label', image_dataset_sub15[6000]['Label'])

In [0]:
vgg16Net = models.vgg16(pretrained=True) 
vgg16Net.classifier  = nn.Sequential(nn.Linear(512 * 7 * 7, 4096))
layer = vgg16Net._modules.get('classifier')
vgg16Net.eval()

In [0]:

# #let try this features thing
def get_vectors(loader, model):
    print('changed')
    all_features = []
    all_labels = []

    for i_batch, data in tqdm(enumerate(loader), total=len(loader)):

        inputs, labels = data['image'].cuda(), data['Label'].cuda()
        my_embedding = torch.zeros(len(inputs),4096)
        
        def copy_data(m, i, o):
            my_embedding.copy_(o.data)
        
        h = layer.register_forward_hook(copy_data)

        model(inputs.cuda())
        h.remove()

        all_features.append(my_embedding)
        all_labels.append(data['Label'][0])

        print(data['Label'])

    return all_features,all_labels



In [0]:
vgg16Net = vgg16Net.cuda()
vec, vec1 = get_vectors(image_loader, vgg16Net)

In [28]:
print(vec[0].shape)

torch.Size([25, 4096])


***
***
## **Problem 2.** Modelling

In [0]:
#SPLIT THE DATA INTO TRAIN AND TEST
#vec - all features
#vec, vec1
split_id = int(0.7 * len(vec))
train_features, test_features = vec[:split_id], vec[split_id:]
train_labels, test_labels = vec1[:split_id], vec2[split_id:]

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

all_data = TensorDataset(torch.stack(vec), torch.stack(vec1))
train_data = TensorDataset(torch.stack(train_features), torch.stack(train_labels))
test_data = TensorDataset(torch.stack(test_features), torch.stack(test_labels))


In [0]:
batch_size = 5

all_loader = DataLoader(all_data, shuffle=True, batch_size=batch_size)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [140]:
train_data[0][0].shape

torch.Size([25, 4096])

In [0]:
class MyNet(nn.Module):
    #frames = 25
    #output_size = 1
    # embedding_dim = 4096

    def __init__(self, frames, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__()

        print('--inti--')
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
    
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        print(x.shape)
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden

    def hidden(self, batch_size):
        weight = next(self.parameters()).data
        # hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
        #               weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

In [165]:
frames = 25+1
output_size = 1
embedding_dim = 4096
hidden_dim = 128
n_layers = 2

model = MyNet(frames, output_size, embedding_dim, hidden_dim, n_layers)
# model.cuda()

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

--inti--


In [0]:
epochs = 10
#start at infinity
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.hidden(batch_size)
    counter = 0
    running_loss = 0

    for inputs, labels in all_loader:

        if(inputs.shape[0] != batch_size):
            print('Validation - Input Shape Issue:',inputs.shape)
            continue

        counter += 1
        h = tuple([e.data for e in h])
        # inputs, labels = inputs.cuda(), labels.cuda()
        # inputs, labels = inputs, labels
        model.zero_grad()

        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()

        running_loss += loss.item()
        
        if counter%10 == 0:
            print('LOSS', counter + 1, running_loss / 10)
            running_loss = 0.0


* ##### **Print the size of your training and test data**

In [152]:
# Don't hardcode the shape of train and test data
#should be 169 and 73 - total = 242
print('Len of training data is :', len(train_features))
print('Len of test/validation data is :', len(test_features))

Shape of training data is : 169
Shape of test/validation data is : 73


---
---
## **Problem 3.** Evaluation

In [0]:

test_losses = []
num_correct = 0

h = model.hidden(batch_size)

model.eval()
for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs, labels
    output, h = model(inputs, h)
    
    test_loss = criterion(output.squeeze(), labels)
    test_losses.append(test_loss.item())

    pred = torch.round(output.squeeze()) 

    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: " np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: %"test_acc*100)

## **Bonus**


* ##### **Print the size of your training and test data**

In [0]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :', )
print('Shape of test/validation data is :', )

* ##### **Modelling and evaluation**

In [0]:
#Write your code for modelling and evaluation

## Submission
---
**Runnable source code in ipynb file and a pdf report are required**.

The report should be of 3 to 4 pages describing what you have done and learned in this homework and report performance of your model. If you have tried multiple methods, please compare your results. If you are using any external code, please cite it in your report. Note that this homework is designed to help you explore and get familiar with the techniques. The final grading will be largely based on your prediction accuracy and the different methods you tried (different architectures and parameters).

Please indicate clearly in your report what model you have tried, what techniques you applied to improve the performance and report their accuracies. The report should be concise and include the highlights of your efforts.
The naming convention for report is **Surname_Givenname_SBUID_report*.pdf**

When submitting your .zip file through blackboard, please
-- name your .zip file as **Surname_Givenname_SBUID_hw*.zip**.

This zip file should include:
```
Surname_Givenname_SBUID_hw*
        |---Surname_Givenname_SBUID_hw*.ipynb
        |---Surname_Givenname_SBUID_hw*.pdf
        |---Surname_Givenname_SBUID_report*.pdf
```

For instance, student Michael Jordan should submit a zip file named "Jordan_Michael_111134567_hw5.zip" for homework5 in this structure:
```
Jordan_Michael_111134567_hw5
        |---Jordan_Michael_111134567_hw5.ipynb
        |---Jordan_Michael_111134567_hw5.pdf
        |---Jordan_Michael_111134567_report*.pdf
```

The **Surname_Givenname_SBUID_hw*.pdf** should include a **google shared link**. To generate the **google shared link**, first create a folder named **Surname_Givenname_SBUID_hw*** in your Google Drive with your Stony Brook account. 

Then right click this folder, click ***Get shareable link***, in the People textfield, enter two TA's emails: ***bo.cao.1@stonybrook.edu*** and ***sayontan.ghosh@stonybrook.edu***. Make sure that TAs who have the link **can edit**, ***not just*** **can view**, and also **uncheck** the **Notify people** box.

Colab has a good feature of version control, you should take advantage of this to save your work properly. However, the timestamp of the submission made in blackboard is the only one that we consider for grading. To be more specific, we will only grade the version of your code right before the timestamp of the submission made in blackboard. 

You are encouraged to post and answer questions on Piazza. Based on the amount of email that we have received in past years, there might be dealys in replying to personal emails. Please ask questions on Piazza and send emails only for personal issues.

Be aware that your code will undergo plagiarism check both vertically and horizontally. Please do your own work.